In [1]:
import pandas as pd
import seaborn as sns
from ipyleaflet import *
from branca.colormap import linear
import json
import pandas
import geopandas as gpd
from colour import Color
from pandas_profiling import ProfileReport
import numpy as np
from ipywidgets import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 1989-01-01 and 2020-12-31
df = pd.read_csv(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/ged211.csv')
#df = pd.read_csv('/Users/mcolas/Documents/ged211.csv')
#df = pd.read_csv('../UCDP.csv')

# On récupére une base de donnée comprenant une liste de données sur les pays du monde
countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

df_map = df[['id','year', 'where_coordinates', 'latitude', 'longitude', 'country','best','type_of_violence', 'side_a', 'side_b', 'source_headline']]

In [3]:
#Construction d'une carte interactive permettant d'obtenir une vision globale du nombre de conflits par état et par année

# Couleurs dans l'ordre croissant 0-10, 10-30, 30-50,50-100,100-400,400- 
def colors_count(count):
    
    colors = ['#f1c40f','#f39c12','#e67e22','#d35400','#e74c3c','#c0392b']
    if count < 11:
        return colors[0]
    elif count < 31:
        return colors[1]
    elif count < 51: 
        return colors[2]
    elif count < 101:
        return colors[3]
    elif count < 401:
        return colors[4]
    else:
        return colors[5]

def map_density(year=1989,type_event=0):

    densityMap = Map(basemap=basemaps.CartoDB.Positron,zoom=2, close_popup_on_click=False)
    
    if type_event == 0:
        data_year = df_map[df_map['year'] == year].country.value_counts()
    else:
        data_year = df_map[df_map['year'] == year][df_map['type_of_violence'] == type_event].country.value_counts()
    
    # On inclue une légende
    legend = LegendControl({'Entre 0 et 10 événements':'#f1c40f', 
                            'Entre 11 et 30 événements':'#f39c12', 
                            'Entre 31 et 50 événements':'#e67e22',
                           'Entre 51 et 100 événements' : '#d35400',
                           'Entre 101 et 400 événements' :'#e74c3c',
                           'Plus de 400 événements' :'#c0392b'}, name="Légende", position="bottomright")
    densityMap.add_control(legend)

    # On affiche la couleur adapté au nombre d'événement 
    for country,count in data_year.iteritems():
        geo_country = countries[countries["name"] == country]
        geodata = GeoData(geo_dataframe = geo_country, 
     style={"color": "black", "fillColor":colors_count(count), "opacity":0.03, "weight":1.9, "dashArray":"2", 
            "fillOpacity":0.6},name = "Countries")
        
        densityMap.add_layer(geodata)
    
    return densityMap

# Création des widgets pour rendre la carte interactive
interact(map_density,
         year=widgets.Dropdown(options=[i for i in range(1989,2021,1)], description='Année : '),
         type_event=widgets.Dropdown(options=[('Tous les événements', 0),
                                              ('Conflit impliquant au moins un état', 1), 
                                              ('Conflit entre des entités non-étatiques', 2), 
                                              ('Conflit visant des civils', 3)], description='Type :'))

interactive(children=(Dropdown(description='Année : ', options=(1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996…

<function __main__.map_density(year=1989, type_event=0)>

In [4]:
#Création d'une carte interactive permettant d'obtenir une vision globale du nombre de morts par état et par année
#La carte permet de connaitre la localisation du conflit et le nombre de morts lié à chaque type de conflit

# Icone en forme d'éclair pour les conflits impliquant deux groupes
# Icone en forme de bombe pour les attaques visant les civils
icon = [AwesomeIcon(name='flash', icon_color='white'),
        AwesomeIcon(name='flash', icon_color='white'),
        AwesomeIcon(name='bomb', icon_color='white')]


# Couleurs dans l'ordre croissant
def colors_fatalities(fatalities):
    
    colors = ['green', 'orange','red','darkred','black']
    
    if fatalities == 0:
        return colors[0]
    elif fatalities < 11:
        return colors[1]
    elif fatalities < 41:
        return colors[2]
    elif fatalities < 101: 
        return colors[3]
    else:
        return colors[4]


    
def map_event(year=1,type_event=0,number_event=0.1,country='Tous'):

    clusterMap = Map(basemap=basemaps.CartoDB.Positron,zoom=2, close_popup_on_click=False)
    
    # On inclue une légende
    legend = LegendControl({'Aucun blessé':'green', 
                            'Entre 0 et 10 décés ':'orange', 
                            'Entre 11 et 40 décés':'red',
                           'Entre 41 et 100 décés' : 'darkred',
                           'Plus de 100 décés' :'black'}, name="Légende", position="bottomright")
    clusterMap.add_control(legend)
    
    # On stocke tous les marqueurs dans une liste
    markers_custom=[]
    
    # On sélectionne le pays si sélectionné
    if country != 'Tous':
        data_temp = df_map[df_map['year'] == year][df_map['country'] == country]
    else: 
        data_temp = df_map[df_map['year'] == year]
    
    if type_event == 0:
        data = data_temp.sample(frac=number_event)
    else:
        data = data_temp[df_map['type_of_violence'] == type_event].sample(frac=number_event)
    
    # On crée tous les marqueurs 
    for index,event in data.iterrows():

        # Création du message qui s'affiche en cliquant sur l'icone
        message = HTML()
        message.value = "<b>Localisation :</b> " + str(event.where_coordinates) + "<br /> <b>Nombre de morts :</b> " + str(event.best)
        if event.type_of_violence == 3:
            message.value += "<br /> <b> Attaque effectuée par : </b>" + event.side_a
            current_icon = AwesomeIcon(name='bomb', marker_color=colors_fatalities(event.best))
        else:
            message.value += "<br /> <b> Entité impliquée : </b>" + event.side_a + " / " + event.side_b
            current_icon = AwesomeIcon(name='flash', marker_color=colors_fatalities(event.best))
        
        
        # Création du marqueur au bon endroit avec la bonne icone
        marker = Marker(location=(event.latitude,event.longitude), draggable=False, icon=current_icon)

        
        marker.popup = message
    
        markers_custom.append(marker)
    
    # On affiche tous les événements sous forme de cluster
    clusterMap.add_layer(MarkerCluster(markers=markers_custom))
    
    return clusterMap

# Création des widgets pour rendre la carte interactive
interact(map_event,
         year=widgets.Dropdown(options=[i for i in range(1989,2021,1)], description='Année : '),
         type_event=widgets.Dropdown(options=[('Tous les événements', 0),
                                              ('Conflit impliquant au moins un état', 1), 
                                              ('Conflit entre des entités non-étatiques', 2), 
                                              ('Conflit visant des civils', 3)], description='Type :'),
         number_event=widgets.Dropdown(options=[('10% des événements de l\'année', 0.1),
                                                ('30% des événements de l\'année', 0.3), 
                                                ('50% des événements de l\'année (possiblement long)', 0.5), 
                                                ('80% des événements de l\'année (possiblement long)', 0.8),
                                               ('100% des événements de l\'année (possiblement long)', 1)], description='Afficher : '),
         country=widgets.Dropdown(options=['Tous'] + [name for name in countries.name.values], description='Pays : '))

interactive(children=(Dropdown(description='Année : ', options=(1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996…

<function __main__.map_event(year=1, type_event=0, number_event=0.1, country='Tous')>